In [1]:
import pandas as pd 
import numpy as np 
import os
import re
import matplotlib.pyplot as plt
from Bio import SeqIO
from itertools import groupby
import seaborn as sns
currdir = os.getcwd()
parent = os.path.dirname(currdir)
gparent = os.path.dirname(parent)

In [2]:
def ensure_dirs_exists(path):
    if "." in path:
	    path = os.path.dirname(path)
    if not os.path.exists(path):
        os.makedirs(path)
    return

In [3]:
## read the module parittion file in data/
mfile = f"../10415/10415_unbiased_normalized_041124.xlsx"
data = pd.read_excel(mfile, sheet_name='Normalized Data')
# pfile = f"{gparent}/example_data/rawdata/data_10297JBK/10297_SupplementalData_Proteome_122723.xlsx"
# prot = pd.read_excel(pfile, sheet_name='Imputed Precursor Data')

In [4]:
data.head()

,Accession,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,112765 Fxn10_Rxn Normalized,...,Fxn5_Rxn_CV,Fxn6_Control_CV,Fxn6_Rxn_CV,Fxn7_Control_CV,Fxn7_Rxn_CV,Fxn8_Control_CV,Fxn8_Rxn_CV,Fxn9_Control_CV,Fxn9_Rxn_CV,SPQC_CV
0,A0A087WUL8;B4DH59;P0DPF2;P0DPF3;Q3BBV2;Q5TAG4;...,Neuroblastoma breakpoint family member 19;Neur...,NBPF19;NBPF26;NBPF20;NBPF9;NBPF8;NBPF12;NBPF14...,1,-.-.-.-.1.-.-.-.-.-.-.-.-.-.1.-.-.-.-.-.-.-.1....,3.784938,1.454183,1.319021,15.777997,356.277658,...,135.0,78.0,166.0,72.9,64.6,166.0,165.0,7.94,29.1,143.0
1,A0A096LP01,"Small integral membrane protein 26, org=Homo s...",SMIM26,1,-.-.-.-.-.-.-.1.1.-.1.-.1.-.-.-.-.-.-.-.-.1.-....,6.076793,1.830363,1.293179,16.744207,1.311561,...,103.0,57.9,151.0,39.4,104.0,25.6,103.0,53.90,62.7,81.8
2,A0A0B4J2A2,"Peptidyl-prolyl cis-trans isomerase A-like 4C,...",PPIAL4C,3,2.3.3.3.2.-.-.-.-.-.-.2.1.3.-.-.-.3.-.-.3.2.2....,6328.085778,6863.235921,11863.451674,14607.199437,7466.690684,...,25.8,147.0,37.5,48.0,22.8,36.7,13.2,1.26,29.7,146.0
3,A0A0B4J2D5;P0DPI2,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,25,7.8.9.6.8.9.20.22.21.20.21.21.13.15.17.15.18.1...,6250.612036,5540.835072,9565.084648,3806.866861,4156.931239,...,67.4,19.6,20.7,55.0,21.8,47.7,16.6,5.37,20.4,45.1
4,A0A0B4J2F0,"Protein PIGBOS1, org=Homo sapiens",PIGBOS1,1,-.-.-.-.-.-.1.1.1.1.1.1.-.1.1.1.1.1.-.1.1.-.-....,4.903224,2.804773,1.222221,2.904877,3.163710,...,62.3,53.0,156.0,62.3,118.0,53.8,166.0,71.00,28.6,17.4


In [5]:
## map mouse kinases
kinomefile = f"/home/poojaparameswaran/Documents/generalData/data/pkinfam.csv"
allkinome = pd.read_csv(kinomefile, index_col=0)
## compress into triples. 
humans = {}
for i, r in allkinome.iterrows():
    humans[r['HumanID']] = (r['HumanName'], r['ProteinName'])
humans

{'P31749': ('AKT1_HUMAN', 'AKT1'),
 'P31751': ('AKT2_HUMAN', 'AKT2'),
 'Q9Y243': ('AKT3_HUMAN', 'AKT3'),
 'Q5VT25': ('MRCKA_HUMAN', 'CDC42BPA'),
 'Q9Y5S2': ('MRCKB_HUMAN', 'CDC42BPB'),
 'Q6DT37': ('MRCKG_HUMAN', 'CDC42BPG'),
 'O14578': ('CTRO_HUMAN', 'CIT'),
 'Q09013': ('DMPK_HUMAN', 'DMPK'),
 'Q15835': ('GRK1_HUMAN', 'GRK1'),
 'P25098': ('ARBK1_HUMAN', 'GRK2'),
 'P35626': ('ARBK2_HUMAN', 'GRK3'),
 'P32298': ('GRK4_HUMAN', 'GRK4'),
 'P34947': ('GRK5_HUMAN', 'GRK5'),
 'P43250': ('GRK6_HUMAN', 'GRK6'),
 'Q8WTQ7': ('GRK7_HUMAN', 'GRK7'),
 'O95835': ('LATS1_HUMAN', 'LATS1'),
 'Q9NRM7': ('LATS2_HUMAN', 'LATS2'),
 'Q9Y2H9': ('MAST1_HUMAN', 'MAST1'),
 'Q6P0Q8': ('MAST2_HUMAN', 'MAST2'),
 'O60307': ('MAST3_HUMAN', 'MAST3'),
 'O15021': ('MAST4_HUMAN', 'MAST4'),
 'Q96GX5': ('GWL_HUMAN', 'MASTL'),
 'O15530': ('PDPK1_HUMAN', 'PDPK1'),
 'Q6A1A2': ('PDPK2_HUMAN', 'PDPK2P'),
 'Q16512': ('PKN1_HUMAN', 'PKN1'),
 'Q16513': ('PKN2_HUMAN', 'PKN2'),
 'Q6P5Z2': ('PKN3_HUMAN', 'PKN3'),
 'P17612': ('KAPCA_HUM

In [6]:
# ## if u need fasta from uniprot 
# ggparent = os.path.dirname(gparent)
# g2parent = os.path.dirname(ggparent)
# all_uniprots_file = f"/home/poojaparameswaran/Documents/generalMouseData/uniprot_sprot.fasta"
# fasta_dict = {fasta.id.split("|")[1]: str(fasta.seq) for fasta in 
#               SeqIO.parse(all_uniprots_file, "fasta")}


**imputed only shows control and rxn. NOT control_CV or spqc_C, will drop those now ->**

In [7]:
data.columns
todrop = [x for x in data.columns if '_CV' in x]
if any('_CV' in col for col in data.columns):
    data.drop(columns=todrop, inplace=True)
data

,Accession,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,112765 Fxn10_Rxn Normalized,...,112757 Fxn9_Control Normalized,112771 Fxn9_Control Normalized,112750 Fxn9_Rxn Normalized,112764 Fxn9_Rxn Normalized,112778 Fxn9_Rxn Normalized,112780_1 SPQC Normalized,112780_2 SPQC Normalized,112780_3 SPQC Normalized,112780_4 SPQC Normalized,112780_5 SPQC Normalized
0,A0A087WUL8;B4DH59;P0DPF2;P0DPF3;Q3BBV2;Q5TAG4;...,Neuroblastoma breakpoint family member 19;Neur...,NBPF19;NBPF26;NBPF20;NBPF9;NBPF8;NBPF12;NBPF14...,1,-.-.-.-.1.-.-.-.-.-.-.-.-.-.1.-.-.-.-.-.-.-.1....,3.784938e+00,1.454183e+00,1.319021e+00,1.577800e+01,3.562777e+02,...,4.342347e+02,3.937723e+02,4.902509e+02,3.214013e+02,2.922193e+02,1.642709e+02,3.175965e+00,3.132719e+02,1.663741e+00,4.025267e+00
1,A0A096LP01,"Small integral membrane protein 26, org=Homo s...",SMIM26,1,-.-.-.-.-.-.-.1.1.-.1.-.1.-.-.-.-.-.-.-.-.1.-....,6.076793e+00,1.830363e+00,1.293179e+00,1.674421e+01,1.311561e+00,...,1.720205e+00,3.548736e+00,2.910903e+00,8.821535e+00,3.665938e+00,1.693638e+00,4.448527e+00,6.242559e+00,1.650346e+01,1.999834e+01
2,A0A0B4J2A2,"Peptidyl-prolyl cis-trans isomerase A-like 4C,...",PPIAL4C,3,2.3.3.3.2.-.-.-.-.-.-.2.1.3.-.-.-.3.-.-.3.2.2....,6.328086e+03,6.863236e+03,1.186345e+04,1.460720e+04,7.466691e+03,...,1.006234e+04,1.013114e+04,4.910616e+03,8.900539e+03,6.490863e+03,4.915811e+03,2.112756e+03,2.982893e+03,9.688014e+02,2.823239e+04
3,A0A0B4J2D5;P0DPI2,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,25,7.8.9.6.8.9.20.22.21.20.21.21.13.15.17.15.18.1...,6.250612e+03,5.540835e+03,9.565085e+03,3.806867e+03,4.156931e+03,...,5.050561e+03,5.537900e+03,1.151466e+03,1.261447e+03,1.677783e+03,2.042241e+04,4.696727e+04,1.954477e+04,2.010140e+04,2.348825e+04
4,A0A0B4J2F0,"Protein PIGBOS1, org=Homo sapiens",PIGBOS1,1,-.-.-.-.-.-.1.1.1.1.1.1.-.1.1.1.1.1.-.1.1.-.-....,4.903224e+00,2.804773e+00,1.222221e+00,2.904877e+00,3.163710e+00,...,2.119037e+01,1.830803e+01,8.448881e+00,6.742310e+00,1.179136e+01,5.833222e+02,3.831807e+02,4.610674e+02,5.622259e+02,4.388652e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9080,Q9Y6X8,"Zinc fingers and homeoboxes protein 2, org=Hom...",ZHX2,1,1.1.-.1.-.1.1.1.-.1.-.1.1.1.1.-.-.-.1.1.-.1.1....,5.594128e+02,4.654511e+02,5.052212e+02,3.970853e+02,3.176195e+02,...,4.121460e+02,3.275647e+02,8.014732e+00,1.566442e+01,1.601950e+00,3.634218e+02,2.966193e+02,4.543597e+02,2.864931e+02,4.202587e+02
9081,Q9Y6X9,"ATPase MORC2, org=Homo sapiens",MORC2,81,67.72.70.72.71.64.55.39.54.46.47.42.44.57.49.4...,1.767046e+05,2.090979e+05,1.945349e+05,2.024509e+05,1.891897e+05,...,2.945053e+05,2.737444e+05,3.086615e+05,3.352921e+05,2.742282e+05,1.357495e+05,1.366888e+05,1.342858e+05,1.379510e+05,1.383161e+05
9082,Q9Y6Y0,"Influenza virus NS1A-binding protein, org=Homo...",IVNS1ABP,50,43.48.41.41.39.42.33.25.30.25.33.25.30.36.34.3...,2.530550e+05,1.973656e+05,2.292746e+05,2.254544e+05,2.353635e+05,...,9.523948e+04,1.035334e+05,1.076963e+05,9.737790e+04,1.020413e+05,9.192811e+04,9.052627e+04,8.724585e+04,1.098707e+05,9.304189e+04
9083,Q9Y6Y8,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,77,49.55.47.55.48.55.56.49.55.51.49.46.49.62.56.5...,1.619100e+05,1.313689e+05,1.467662e+05,1.364597e+05,1.680148e+05,...,1.895040e+05,1.750872e+05,1.943410e+05,2.265157e+05,2.409775e+05,2.285799e+05,2.377126e+05,2.500703e+05,2.724666e+05,2.908661e+05


In [8]:
def remove_imputed(row, columns):
    datacols = columns[columns.index('Detected_Imputed')+1:]
    dimp = [int(x) if x.isdigit() else x for x in list(row['Detected_Imputed'].replace(".", " ").split(' '))]
    rdata = row.copy()
    for i, imp in enumerate(dimp):
        if imp== '-':
            rdata[datacols[i]] = np.NaN
    return rdata

# s1 = data.apply(lambda row: remove_imputed(row, list(data.columns)), axis=1)
s1 = data.copy(deep=True)
s1.head(), s1.shape

(                                           Accession  \
 0  A0A087WUL8;B4DH59;P0DPF2;P0DPF3;Q3BBV2;Q5TAG4;...   
 1                                         A0A096LP01   
 2                                         A0A0B4J2A2   
 3                                  A0A0B4J2D5;P0DPI2   
 4                                         A0A0B4J2F0   
 
                                          Description  \
 0  Neuroblastoma breakpoint family member 19;Neur...   
 1  Small integral membrane protein 26, org=Homo s...   
 2  Peptidyl-prolyl cis-trans isomerase A-like 4C,...   
 3  Putative glutamine amidotransferase-like class...   
 4                  Protein PIGBOS1, org=Homo sapiens   
 
                                                Genes  Peptides  \
 0  NBPF19;NBPF26;NBPF20;NBPF9;NBPF8;NBPF12;NBPF14...         1   
 1                                             SMIM26         1   
 2                                            PPIAL4C         3   
 3                                       GAT

In [9]:
if 'Human_Kinase' not in s1.columns:
    s1.insert(s1.columns.get_loc('Accession')+1, 'Human_Kinase', '')
for i, r in s1.iterrows():
    poss = r['Accession'].split(';')
    match = False
    for x in poss:
        if x in humans.keys():
            s1.at[i, 'Human_Kinase'] = humans[x][0]
            match=True
            break
    if not match:
        s1.at[i, 'Human_Kinase'] = False
s1

,Accession,Human_Kinase,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,...,112757 Fxn9_Control Normalized,112771 Fxn9_Control Normalized,112750 Fxn9_Rxn Normalized,112764 Fxn9_Rxn Normalized,112778 Fxn9_Rxn Normalized,112780_1 SPQC Normalized,112780_2 SPQC Normalized,112780_3 SPQC Normalized,112780_4 SPQC Normalized,112780_5 SPQC Normalized
0,A0A087WUL8;B4DH59;P0DPF2;P0DPF3;Q3BBV2;Q5TAG4;...,False,Neuroblastoma breakpoint family member 19;Neur...,NBPF19;NBPF26;NBPF20;NBPF9;NBPF8;NBPF12;NBPF14...,1,-.-.-.-.1.-.-.-.-.-.-.-.-.-.1.-.-.-.-.-.-.-.1....,3.784938e+00,1.454183e+00,1.319021e+00,1.577800e+01,...,4.342347e+02,3.937723e+02,4.902509e+02,3.214013e+02,2.922193e+02,1.642709e+02,3.175965e+00,3.132719e+02,1.663741e+00,4.025267e+00
1,A0A096LP01,False,"Small integral membrane protein 26, org=Homo s...",SMIM26,1,-.-.-.-.-.-.-.1.1.-.1.-.1.-.-.-.-.-.-.-.-.1.-....,6.076793e+00,1.830363e+00,1.293179e+00,1.674421e+01,...,1.720205e+00,3.548736e+00,2.910903e+00,8.821535e+00,3.665938e+00,1.693638e+00,4.448527e+00,6.242559e+00,1.650346e+01,1.999834e+01
2,A0A0B4J2A2,False,"Peptidyl-prolyl cis-trans isomerase A-like 4C,...",PPIAL4C,3,2.3.3.3.2.-.-.-.-.-.-.2.1.3.-.-.-.3.-.-.3.2.2....,6.328086e+03,6.863236e+03,1.186345e+04,1.460720e+04,...,1.006234e+04,1.013114e+04,4.910616e+03,8.900539e+03,6.490863e+03,4.915811e+03,2.112756e+03,2.982893e+03,9.688014e+02,2.823239e+04
3,A0A0B4J2D5;P0DPI2,False,Putative glutamine amidotransferase-like class...,GATD3B;GATD3,25,7.8.9.6.8.9.20.22.21.20.21.21.13.15.17.15.18.1...,6.250612e+03,5.540835e+03,9.565085e+03,3.806867e+03,...,5.050561e+03,5.537900e+03,1.151466e+03,1.261447e+03,1.677783e+03,2.042241e+04,4.696727e+04,1.954477e+04,2.010140e+04,2.348825e+04
4,A0A0B4J2F0,False,"Protein PIGBOS1, org=Homo sapiens",PIGBOS1,1,-.-.-.-.-.-.1.1.1.1.1.1.-.1.1.1.1.1.-.1.1.-.-....,4.903224e+00,2.804773e+00,1.222221e+00,2.904877e+00,...,2.119037e+01,1.830803e+01,8.448881e+00,6.742310e+00,1.179136e+01,5.833222e+02,3.831807e+02,4.610674e+02,5.622259e+02,4.388652e+02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9080,Q9Y6X8,False,"Zinc fingers and homeoboxes protein 2, org=Hom...",ZHX2,1,1.1.-.1.-.1.1.1.-.1.-.1.1.1.1.-.-.-.1.1.-.1.1....,5.594128e+02,4.654511e+02,5.052212e+02,3.970853e+02,...,4.121460e+02,3.275647e+02,8.014732e+00,1.566442e+01,1.601950e+00,3.634218e+02,2.966193e+02,4.543597e+02,2.864931e+02,4.202587e+02
9081,Q9Y6X9,False,"ATPase MORC2, org=Homo sapiens",MORC2,81,67.72.70.72.71.64.55.39.54.46.47.42.44.57.49.4...,1.767046e+05,2.090979e+05,1.945349e+05,2.024509e+05,...,2.945053e+05,2.737444e+05,3.086615e+05,3.352921e+05,2.742282e+05,1.357495e+05,1.366888e+05,1.342858e+05,1.379510e+05,1.383161e+05
9082,Q9Y6Y0,False,"Influenza virus NS1A-binding protein, org=Homo...",IVNS1ABP,50,43.48.41.41.39.42.33.25.30.25.33.25.30.36.34.3...,2.530550e+05,1.973656e+05,2.292746e+05,2.254544e+05,...,9.523948e+04,1.035334e+05,1.076963e+05,9.737790e+04,1.020413e+05,9.192811e+04,9.052627e+04,8.724585e+04,1.098707e+05,9.304189e+04
9083,Q9Y6Y8,False,"SEC23-interacting protein, org=Homo sapiens",SEC23IP,77,49.55.47.55.48.55.56.49.55.51.49.46.49.62.56.5...,1.619100e+05,1.313689e+05,1.467662e+05,1.364597e+05,...,1.895040e+05,1.750872e+05,1.943410e+05,2.265157e+05,2.409775e+05,2.285799e+05,2.377126e+05,2.500703e+05,2.724666e+05,2.908661e+05


In [13]:
s2 = s1[s1['Human_Kinase'] != False]

In [16]:
s1.to_csv(f'{mfile.split(".xlsx")[0]}_AnnotatedHumanKinome.csv')
s2

,Accession,Human_Kinase,Description,Genes,Peptides,Detected_Imputed,112744 Fxn10_Control Normalized,112758 Fxn10_Control Normalized,112772 Fxn10_Control Normalized,112751 Fxn10_Rxn Normalized,...,112757 Fxn9_Control Normalized,112771 Fxn9_Control Normalized,112750 Fxn9_Rxn Normalized,112764 Fxn9_Rxn Normalized,112778 Fxn9_Rxn Normalized,112780_1 SPQC Normalized,112780_2 SPQC Normalized,112780_3 SPQC Normalized,112780_4 SPQC Normalized,112780_5 SPQC Normalized
5,A0A0B4J2F2;P57059,SIK1B_HUMAN,Putative serine/threonine-protein kinase SIK1B...,SIK1B;SIK1,15,10.12.10.10.11.13.11.5.9.10.7.7.9.5.6.10.9.4.5...,8966.244557,7308.774602,8763.373499,9420.749301,...,10948.819020,10442.664055,8384.168996,7871.567683,8049.957273,4257.345850,4038.235115,3309.965378,3857.857478,4024.544669
154,O00141,SGK1_HUMAN,"Serine/threonine-protein kinase Sgk1, org=Homo...",SGK1,7,6.5.6.6.5.4.4.2.4.3.1.4.2.6.4.4.4.5.2.3.3.3.3....,15870.258819,10343.783089,11770.911566,12676.190954,...,4345.729843,3979.979593,5989.902286,6132.303425,4986.231928,13326.720659,14484.834112,14754.400029,14078.329655,15085.227849
180,O00238,BMR1B_HUMAN,"Bone morphogenetic protein receptor type-1B, o...",BMPR1B,1,-.-.-.-.-.-.1.-.-.-.-.-.1.-.1.1.1.1.-.-.-.-.-....,8.457218,13.655893,1.541628,14.093329,...,1.818127,1.579827,12.784533,1.266578,2.032145,2.467139,19.053053,216.080022,1.102350,14.277163
195,O00311,CDC7_HUMAN,"Cell division cycle 7-related protein kinase, ...",CDC7,30,26.25.24.23.25.20.13.9.13.8.10.13.13.16.12.14....,35625.073555,29760.044580,33417.690964,35242.215973,...,32171.449267,32483.607996,33642.150254,32934.689033,32361.189431,17705.582224,20041.352882,18447.772206,16591.349876,18826.094684
197,O00329,PK3CD_HUMAN,"Phosphatidylinositol 4,5-bisphosphate 3-kinase...",PIK3CD,13,8.7.6.10.11.8.7.3.5.3.6.6.6.5.7.10.5.6.6.7.6.9...,3381.358547,2288.085433,2117.901021,4732.677458,...,3509.196311,3510.079463,4743.315554,5424.786788,4532.702194,2301.926871,2703.181597,2658.167482,2596.690570,2624.359482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8924,Q9Y4K4,M4K5_HUMAN,Mitogen-activated protein kinase kinase kinase...,MAP4K5,21,16.15.13.14.15.14.12.7.9.11.8.9.6.11.7.12.7.12...,13290.864306,11682.162395,13382.112879,14705.707618,...,14094.644205,16354.014794,20423.038583,19088.172099,18369.150330,7035.104099,5946.420461,5442.133532,6170.086703,7734.813663
8986,Q9Y5S2,MRCKB_HUMAN,"Serine/threonine-protein kinase MRCK beta, org...",CDC42BPB,175,137.140.139.151.150.132.105.95.113.95.101.103....,436571.087061,395162.409438,432232.372014,444456.844198,...,601865.591490,636537.650787,725976.486203,712959.774901,706456.291749,265263.678621,320946.890808,274169.274857,255825.390457,340932.803895
9042,Q9Y6E0,STK24_HUMAN,"Serine/threonine-protein kinase 24, org=Homo s...",STK24,47,42.37.40.40.41.41.39.36.38.37.39.41.36.38.40.4...,314606.575744,266991.609750,270471.071203,293154.479325,...,236578.315930,243069.406478,247704.766118,254244.277780,241093.271670,244854.431259,271884.358528,251863.656688,258097.237360,274053.078172
9060,Q9Y6M4,KC1G3_HUMAN,"Casein kinase I isoform gamma-3, org=Homo sapiens",CSNK1G3,32,5.8.4.5.5.7.24.26.27.22.23.22.25.29.29.28.26.2...,2369.575464,2241.742862,3011.563308,2519.508701,...,5582.032281,5990.611244,5753.109557,9086.667031,6626.673697,33962.773023,34142.149982,34165.284929,32577.598952,38824.414954


In [15]:
s2.loc[180]

Accession                                                                     O00238
Human_Kinase                                                             BMR1B_HUMAN
Description                        Bone morphogenetic protein receptor type-1B, o...
Genes                                                                         BMPR1B
Peptides                                                                           1
Detected_Imputed                   -.-.-.-.-.-.1.-.-.-.-.-.1.-.1.1.1.1.-.-.-.-.-....
112744 Fxn10_Control Normalized                                             8.457218
112758 Fxn10_Control Normalized                                            13.655893
112772 Fxn10_Control Normalized                                             1.541628
112751 Fxn10_Rxn Normalized                                                14.093329
112765 Fxn10_Rxn Normalized                                                 3.217365
112779 Fxn10_Rxn Normalized                                      

In [36]:
## plot across fractions now.
todrop = [x for x in s2.columns if ('_Control' in x) |( 'SPQC' in x)]
if any(x in s2.columns for x in todrop):
    s3 = s2.drop(columns=todrop).reset_index(drop=True)

def plot_k(ax, xvals, y_avg, kinName):
    xvals = sorted(xvals, key=lambda v: int(v.split('Fxn')[-1]))
    ax.plot(xvals, y_avg)
    ax.set_title(f'Human Kinase: {kinName}')
    ax.set_xlabel('Fractions')
    ax.set_ylabel('Abundance')
    plt.tight_layout()
    
def get_AVGabundance(row): # row is a series
    ind = list(row.index).index('Detected_Imputed')
    datacols = list(row.index[ind+1:])
    running_avg = []
    xa = []
    for j, i in groupby(datacols, lambda a: a.split('_')[0].split(' ')[-1]):
        xa.append(j)
        rel = list(row[list(i)].values)
        if (any(np.isnan(rel))) is False: # is nan? = true, so not nan
            running_avg.append(np.mean(rel))
        else:
            running_avg.append(0)
    xa = sorted(xa, key=lambda v: int(v.split('Fxn')[-1]))

    return xa, running_avg, row['Human_Kinase']

v = pd.DataFrame()

for i, row in s3.iterrows():
    x, vals, kinase = (get_AVGabundance(row))
    zipped = dict(zip(x, vals))
    if i == 0:
        v = pd.DataFrame(columns = x)
    v.loc[kinase] = zipped
v

,Fxn4,Fxn5,Fxn6,Fxn7,Fxn8,Fxn9,Fxn10
SIK1B_HUMAN,17638.947947,3991.192866,3170.884011,4464.801373,3707.325416,16038.794565,8101.897984
SGK1_HUMAN,11880.631111,10734.183560,24372.564078,8738.091481,16691.194919,16921.659668,5702.812546
BMR1B_HUMAN,0.000000,0.000000,437.428765,0.000000,0.000000,0.000000,0.000000
CDC7_HUMAN,32966.980252,14381.098318,14159.373900,14156.861639,18863.846073,17087.319377,32979.342906
PK3CD_HUMAN,5530.624876,2131.047549,2073.916118,2782.764015,2936.807392,3785.950135,4900.268178
...,...,...,...,...,...,...,...
M4K5_HUMAN,13070.839347,2888.188071,3910.122676,5418.652693,3183.668704,7302.633709,19293.453671
MRCKB_HUMAN,403128.869522,191026.731152,220901.459992,191876.432063,255215.923023,333638.376664,715130.850951
STK24_HUMAN,280892.683340,351516.559945,326042.063846,309042.754663,316251.888313,234189.548415,247680.771856
KC1G3_HUMAN,2559.988078,69038.528447,91012.331034,46448.917366,32343.310551,34283.954847,7155.483428


In [50]:
## write to excel sheet sorting 
SAVE_FILE = f"{os.path.dirname(mfile)}/FractionatedKinases_{os.path.basename(mfile)}"
with pd.ExcelWriter(SAVE_FILE, engine = 'xlsxwriter') as writer:
    for c in v.columns:
        now = (v[[c]].sort_values(by=c, ascending=False))
        now.to_excel(writer, sheet_name = c)
        worksheet = writer.sheets[c]

        # Get the length of the longest item in each column and set the column width
        max_length = max((now[c].astype(str).map(len).max(),  # length of the longest item
                          len(str(c))))  # length of the column header
        worksheet.set_column(0, 0, max_length + 1)  # set column width

In [ ]:
# plotperimg = 10
# fig, axes = plt.subplots(nrows=int(plotperimg/2), ncols=2, figsize=(20, 20))
# axes = axes.flatten()
# for i, row in s3.iterrows(): # each row is a series 
#     if i % plotperimg == 0 and i != 0:
#         plt.tight_layout()
#         plt.savefig(f"../plots/Plot{i}_KinaseFractionation")
#         plt.show()
#         fig, axes = plt.subplots(nrows=int(plotperimg/2), ncols=2, figsize=(20, 20))
#         axes = axes.flatten()
#     xa, run_avg, hkin = get_AVGabundance(row)
#     plot_k(axes[i % plotperimg], xa, run_avg, hkin)